In [1]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np

from scipy import stats
from numpy.random import Generator, PCG64

import tqdm
import json
import inspect
import warnings
from IPython.display import display, Markdown, Latex
import numpy as np


/var/folders/x6/7z03llzd6tv506jmg9k6xsxc0000gn/T/ipykernel_32568/443391096.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

##### Config

In [3]:
pd.set_option("display.max_columns", None)

In [4]:
def get_sample_from_distribution(n_sample, random_state, params, expected=False):
    """
    Return a sample of size N from de distribution
    """
    scale, loc, b, a = params["scale"], params["loc"], params["b"], params["a"]
    #logger.info(f"param a: {a} \n param b: {b} \n param loc: {loc} \n param scale: {scale}")
    distribution = stats.johnsonsu(a=a, b=b, loc=loc, scale=scale)
    #distribution.random_state = random_state
    if expected:
        mean = distribution.expect()
        return mean if not pd.isnull(mean) else 0
    return distribution.rvs(size=n_sample) if not pd.isnull(n_sample) else 0

In [5]:
def create_folder(path):
    # Verificar si la carpeta existe
    if not os.path.exists(path):
        os.makedirs(path)
        print(f"La carpeta {path} ha sido creada.")
    else:
        print(f"La carpeta {path} ya existe.")

# Data
## Daily Demand

In [7]:
raw = pd.read_csv("../data/old/output_layer_process.csv")
logger.info(f'Quantity of rows input {len(raw.index)}')

INFO:__main__:Quantity of rows input 71774


In [8]:
display(Markdown(f"#### Quantity of customer by layer-period"))
pd.pivot_table(data=raw, index=["year","month"], columns=["layer"], values="cod_customer", aggfunc="count")

#### Quantity of customer by layer-period

layer            0     1      2
year month                     
2020 1      1421.0   9.0   42.0
     2      1253.0  14.0   46.0
     3      1057.0   4.0   29.0
     4       962.0   6.0   18.0
     5      1138.0   6.0   31.0
     6      1284.0   9.0   48.0
     7      1293.0  14.0   49.0
     8      1189.0   5.0   38.0
     9      1525.0   5.0   56.0
     10     1787.0   7.0   73.0
     11     1685.0  17.0   75.0
     12     2030.0  17.0   93.0
2021 1      1391.0   7.0   82.0
     2       385.0   NaN    9.0
     3      1734.0   3.0   68.0
     4      1683.0   5.0   69.0
     5      1530.0  13.0   64.0
     6      1473.0  11.0   46.0
     7      1732.0  12.0   75.0
     8      1795.0  13.0   71.0
     9      2055.0  13.0   87.0
     10     2339.0  20.0   91.0
     11     1896.0  15.0   97.0
     12     2513.0  26.0  112.0
2022 1      1760.0  17.0   83.0
     2      2495.0  18.0   99.0
     3      2594.0  13.0  107.0
     4      2459.0  13.0   86.0
     5      2214.0  20.0   86.0
     6      2831.0  15.0  127.0
     7      2558.0  24.0   93.0
     8      2534.0  22.0   92.0
     9      2758.0  22.0  128.0
     10     2998.0  26.0  134.0
     11     2853.0  29.0  122.0
     12     3293.0  37.0  144.0

## Params Distribution

In [9]:
f = open('../data/scenarios/distribution_params.json')
params = json.load(f)
#params

## Customer per Pixel-Month related to 2022

In [10]:
display(Markdown(f"Considerando el año 2022, vamos a samplear tantas veces como clientes tengamos en cada pixel, por cada pixel se sampleara size(pixel) * 12 meses"))
pivot_tbl = pd.pivot_table(data=raw[(raw.year==2022)], index=["pixel"], columns=["layer"], values="cod_customer", aggfunc="nunique")
display(pivot_tbl.T)

Considerando el año 2022, vamos a samplear tantas veces como clientes tengamos en cada pixel, por cada pixel se sampleara size(pixel) * 12 meses

pixel,26.0,42.0,71.0,72.0,73.0,87.0,88.0,104.0,105.0,106.0,107.0,108.0,109.0,111.0,115.0,116.0,119.0,120.0,121.0,122.0,123.0,124.0,125.0,126.0,127.0,130.0,131.0,132.0,133.0,134.0,135.0,136.0,137.0,138.0,139.0,140.0,141.0,142.0,143.0,146.0,147.0,148.0,149.0,150.0,151.0,152.0,153.0,154.0,155.0,156.0,157.0,158.0,161.0,162.0,163.0,164.0,165.0,166.0,167.0,168.0,169.0,170.0,171.0,172.0,173.0,177.0,178.0,179.0,180.0,181.0,182.0,183.0,184.0,185.0,188.0,189.0,192.0,193.0,194.0,195.0,196.0,197.0,198.0,199.0,200.0,201.0,208.0,209.0,210.0,211.0,212.0,213.0,214.0,215.0,216.0,217.0,218.0,224.0,226.0,227.0,228.0,229.0,230.0,234.0,235.0,241.0,242.0,244.0,245.0,246.0,258.0,259.0,261.0,262.0,263.0,274.0,275.0,279.0,280.0,290.0,291.0
layer,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,3.0,3.0,12.0,25.0,36.0,10.0,7.0,6.0,1.0,8.0,1.0,11.0,1.0,2.0,11.0,3.0,6.0,59.0,77.0,74.0,43.0,49.0,54.0,22.0,30.0,34.0,56.0,36.0,2.0,1.0,35.0,54.0,21.0,60.0,1.0,5.0,62.0,63.0,11.0,39.0,44.0,12.0,45.0,40.0,47.0,40.0,40.0,21.0,3.0,12.0,8.0,9.0,2.0,26.0,165.0,96.0,40.0,45.0,18.0,20.0,50.0,8.0,7.0,27.0,1.0,25.0,128.0,226.0,171.0,84.0,98.0,81.0,42.0,28.0,4.0,32.0,10.0,96.0,184.0,335.0,132.0,120.0,104.0,65.0,35.0,18.0,57.0,78.0,88.0,109.0,85.0,96.0,66.0,40.0,35.0,6.0,4.0,14.0,48.0,60.0,46.0,117.0,20.0,3.0,1.0,3.0,58.0,4.0,68.0,86.0,30.0,6.0,11.0,25.0,14.0,34.0,3.0,1.0,11.0,12.0,3.0
1,NaN,2.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,3.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,1.0,1.0,NaN,NaN,3.0,NaN,NaN,1.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,1.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,3.0,3.0,NaN,1.0,NaN,1.0,NaN,NaN,1.0,NaN,2.0,2.0,NaN,1.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,NaN,2.0,1.0,1.0,3.0,NaN,1.0,1.0,NaN,1.0,NaN,NaN,NaN,1.0,1.0,1.0,NaN,1.0,7.0,5.0,4.0,4.0,NaN,NaN,4.0,NaN,NaN,2.0,NaN,NaN,2.0,NaN,3.0,5.0,NaN,NaN,6.0,4.0,NaN,6.0,1.0,1.0,2.0,1.0,4.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,1.0,2.0,4.0,5.0,1.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,9.0,7.0,15.0,7.0,4.0,3.0,3.0,1.0,NaN,1.0,2.0,8.0,22.0,41.0,19.0,5.0,7.0,6.0,1.0,NaN,3.0,4.0,4.0,7.0,1.0,4.0,3.0,NaN,1.0,1.0,1.0,NaN,3.0,2.0,1.0,7.0,NaN,NaN,NaN,NaN,2.0,NaN,4.0,5.0,NaN,NaN,NaN,NaN,1.0,4.0,NaN,NaN,1.0,NaN,NaN


## A) Layer Low

In [11]:
LAYER="low"

In [12]:
df_low = pd.DataFrame(pivot_tbl[0]).reset_index()

In [13]:
df_low = pd.DataFrame(pivot_tbl[0]).reset_index()
df_low.rename(columns={
        0: "n_customers",
    }, inplace=True
)
df_low["layer"] = LAYER
df_low.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121 entries, 0 to 120
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   pixel        121 non-null    float64
 1   n_customers  121 non-null    float64
 2   layer        121 non-null    object 
dtypes: float64(2), object(1)
memory usage: 3.0+ KB


In [22]:
# set RANDOM STATE
random_state = Generator(PCG64(12345))
output = []
# sampling by pixel

for _, row in df_low.iterrows():
    id_pixel = f'0-{int(row["pixel"])}'
    stop_by_period = [row['n_customers'] for i in range(12)]
    demand_by_period = [round(sum(
        get_sample_from_distribution(
            n_sample=int(row['n_customers']),
            random_state = random_state,
            params=params['low'][str(period+1)],
            expected=False,
        ))
    ,2) for period in range(12)]

    drop_by_period = list(np.array(demand_by_period) / np.array(stop_by_period))
    
    output.append({
        'id_pixel': id_pixel,
        'stop_by_period':stop_by_period,
        'drop_by_period': drop_by_period, #list(np.array(demand_by_period) / np.array(stop_by_period)),
        'demand_by_period':demand_by_period, #list(np.array(stop_by_period) * np.array(drop_by_period)),
    })

In [23]:
list(filter(lambda x: x['id_pixel']=='0-121',output))

[{'id_pixel': '0-121',
  'stop_by_period': [77.0,
   77.0,
   77.0,
   77.0,
   77.0,
   77.0,
   77.0,
   77.0,
   77.0,
   77.0,
   77.0,
   77.0],
  'drop_by_period': [3.0241558441558443,
   2.6494805194805195,
   2.7733766233766235,
   3.1587012987012986,
   2.716233766233766,
   2.8671428571428574,
   3.869090909090909,
   2.413766233766234,
   2.9284415584415586,
   2.814285714285714,
   2.8987012987012983,
   4.137922077922078],
  'demand_by_period': [232.86,
   204.01,
   213.55,
   243.22,
   209.15,
   220.77,
   297.92,
   185.86,
   225.49,
   216.7,
   223.2,
   318.62]}]

## B) Layer Medium

In [ ]:
LAYER="medium"
df_medium = pd.DataFrame(pivot_tbl[2]).reset_index()
df_medium.rename(columns={
        2: "n_customers",
    }, inplace=True
)
df_medium = df_medium.dropna()
df_medium["layer"] = LAYER

In [ ]:
for _, row in df_medium.iterrows():
    id_pixel = f'1-{int(row["pixel"])}'
    stop_by_period = [row['n_customers'] for i in range(12)]
    drop_by_period = [round(get_sample_from_distribution(
        n_sample=int(row['n_customers']),
        random_state = random_state,
        params=params['medium'][str(period+1)],
        expected=True
    ),2) for period in range(12)]
    output.append({
        'id_pixel': id_pixel,
        'stop_by_period':stop_by_period,
        'drop_by_period':drop_by_period,
        'demand_by_period':list(np.array(stop_by_period) * np.array(drop_by_period)),
    })

## C) Layer high

In [ ]:
LAYER="high"
df_high = pd.DataFrame(pivot_tbl[1]).reset_index()
df_high.rename(columns={
        1: "n_customers",
    }, inplace=True
)
df_high = df_high.dropna()
df_high["layer"] = LAYER

In [ ]:
for _, row in df_high.iterrows():
    id_pixel = f'2-{int(row["pixel"])}'
    stop_by_period = [row['n_customers'] for i in range(12)]
    drop_by_period = [round(get_sample_from_distribution(
        n_sample=int(row['n_customers']),
        random_state = random_state,
        params=params['high'][str(period+1)],
        expected=True
    ),2) for period in range(12)]
    output.append({
        'id_pixel': id_pixel,
        'stop_by_period':stop_by_period,
        'drop_by_period':drop_by_period,
        'demand_by_period':list(np.array(stop_by_period) * np.array(drop_by_period)),
    })

In [ ]:
df_output = pd.DataFrame.from_dict(output)

In [ ]:
df_output.to_excel(f'../../data/scenarios/scenario_expected.xlsx', index=False)

# Creation Multiple-Scenario

In [25]:
N_SCENARIO = 500
LAYERS = ["low", "medium", "high"]

In [27]:
path=f'../data/scenarios'
create_folder(path)

La carpeta ../data/scenarios ya existe.


In [28]:
config_dict = {
    "low":'0',
    'medium':'1',
    'high':'2',
}

In [29]:
pivot_tbl_copy = pivot_tbl.rename(columns={0: "low", 1:"high", 2:"medium"}).copy()
for id in range(N_SCENARIO):
    logger.info(f"[scenario {id}] running...")

    output = []
    for layer in LAYERS:
        df = pd.DataFrame(pivot_tbl_copy[layer]).reset_index().copy()
        df = df.rename(columns={layer: "n_customers"})
        df = df.dropna()
        df['layer'] = layer
        
        for _, row in df.iterrows():
            id_pixel = f'{config_dict[layer]}-{int(row["pixel"])}'
            stop_by_period = [row['n_customers'] for i in range(12)]
            demand_by_period = [round(sum(get_sample_from_distribution(
                n_sample=int(row['n_customers']),
                random_state = random_state,
                params=params[layer][str(period+1)]
            )),2) for period in range(12)]

            drop_by_period = list(np.array(demand_by_period) / np.array(stop_by_period))
            output.append({
                'id_pixel': id_pixel,
                'stop_by_period':stop_by_period,
                'drop_by_period':drop_by_period,
                'demand_by_period':demand_by_period,
            })
    df_output = pd.DataFrame.from_dict(output)
    logger.info(f"[scenario {id}] saving file...")
    df_output.to_excel(f'../data/scenarios/scenario_{id}.xlsx', index=False)

INFO:__main__:[scenario 0] running...
INFO:__main__:[scenario 0] saving file...
INFO:__main__:[scenario 1] running...
INFO:__main__:[scenario 1] saving file...
INFO:__main__:[scenario 2] running...
INFO:__main__:[scenario 2] saving file...
INFO:__main__:[scenario 3] running...
INFO:__main__:[scenario 3] saving file...
INFO:__main__:[scenario 4] running...
INFO:__main__:[scenario 4] saving file...
INFO:__main__:[scenario 5] running...
INFO:__main__:[scenario 5] saving file...
INFO:__main__:[scenario 6] running...
INFO:__main__:[scenario 6] saving file...
INFO:__main__:[scenario 7] running...
INFO:__main__:[scenario 7] saving file...
INFO:__main__:[scenario 8] running...
INFO:__main__:[scenario 8] saving file...
INFO:__main__:[scenario 9] running...
INFO:__main__:[scenario 9] saving file...
INFO:__main__:[scenario 10] running...
INFO:__main__:[scenario 10] saving file...
INFO:__main__:[scenario 11] running...
INFO:__main__:[scenario 11] saving file...
INFO:__main__:[scenario 12] running.

## Test Lectura

In [1]:
from src.etl import Data

In [10]:
pixels = Data.load_scenario('expected',True)

INFO:src.etl:Loading data from pixels from path ../../data/input_pixels.xlsx
INFO:src.etl:Count of PIXELS loaded: 191
INFO:src.etl:Loaded OK
INFO:src.etl:--------------------------------------------------
{
  "lon": -68.07457544268874,
  "lat": -16.58741936964211,
  "id_pixel": "0-42",
  "area_surface": 1,
  "speed_intra_stop": {
    "small": 27.872064777327935,
    "large": 29.759654178674353
  },
  "demand_by_period": [],
  "drop_by_period": [],
  "stop_by_period": [],
  "k": 0.57
}
INFO:src.etl:--------------------------------------------------
{
  "lon": -68.09331807455757,
  "lat": -16.5693467958316,
  "id_pixel": "0-71",
  "area_surface": 3,
  "speed_intra_stop": {
    "small": 26.805120702267736,
    "large": 28.829740361919747
  },
  "demand_by_period": [],
  "drop_by_period": [],
  "stop_by_period": [],
  "k": 0.57
}
INFO:src.etl:--------------------------------------------------
{
  "lon": -68.09331807455756,
  "lat": -16.55579236547371,
  "id_pixel": "0-104",
  "area_surface

In [11]:
pixels['1-162'].__dict__

{'lon': -68.149545970164,
 'lat': -16.51512907440006,
 'id_pixel': '1-162',
 'area_surface': 1,
 'speed_intra_stop': {'small': 20.87547169811321, 'large': 23.17068062827225},
 'demand_by_period': [60.92,
  57.3,
  59.7,
  60.52,
  57.66,
  60.32,
  57.82,
  60.9,
  58.04,
  59.42,
  55.54,
  59.84],
 'drop_by_period': [30.46,
  28.65,
  29.85,
  30.26,
  28.83,
  30.16,
  28.91,
  30.45,
  29.02,
  29.71,
  27.77,
  29.92],
 'stop_by_period': [2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0],
 'k': 0.57}